## Differential Evoloution
- initialize a random population of candidate solutions
- for each individual in population
    - mutate
    - crossover
    - select
    - repeat or exit
- - similar to GA but but performs mutation and crossover on float values (rather than bit strings)
- https://www.sciencedirect.com/science/article/abs/pii/S0378779615001947
- https://pablormier.github.io/2017/09/05/a-tutorial-on-differential-evolution-with-python/#
- https://stackoverflow.com/questions/9506809/whats-differential-evolution-and-how-does-it-compare-to-a-genetic-algorithm
- https://nathanrooy.github.io/posts/2017-08-27/simple-differential-evolution-with-python/
    

In [35]:
import random
import numpy as np

In [83]:
def ObjectiveFunction(x):
    """
        The function to minimize
    """
    # sphere problem http://www.sfu.ca/~ssurjano/spheref.html
    return sum([x[i]**2 for i in range(len(x))])

In [91]:
objectiveFunction = lambda x: sum(x**2)

#### demonstrate objective functions

In [93]:
print(ObjectiveFunction([1,2,3,4]))
print(objectiveFunction(np.array([1,2,3,4])))

30
30


In [4]:
def GeneratePopulation(bounds, numVars, popSize):
    """
        initialize the population where each candidate solution is a vector 
        of length <numVars> whos value is a uniformly distributed random
        number in range <bounds>
    """
    population = []
    # iterate through the population size
    for i in range(0, popSize):
        candidate = []
        # iterate through the number of variables per candidate
        for j in range(0, numVars):
            # generate a random number within the bounds
            candidate.append(random.uniform(bounds[j][0], bounds[j][1]))
        population.append(candidate)
    return population

#### demonstrate GeneratePopulation

In [130]:
bounds = [(-10, 10), (20, 30), (0,3)]
numVars = 3
popSize = 5
mutationFactor = .7
test = GeneratePopulation(bounds, numVars, popSize)
test

[[-2.8120181134382065, 25.94414285995579, 1.944089342325594],
 [9.747930370454469, 24.37128812802513, 1.7645791838718703],
 [8.150653108839833, 25.532830633865853, 0.5189086154466039],
 [-7.19070963296746, 24.647570443300864, 1.7369580410077832],
 [-9.028871391149938, 27.871870464845124, 0.8390320621524372]]

In [10]:
def CheckBounds(candidate, bounds):
    """
        after mutation, it is possible for a candidate variable to be outside
        of the bounds. This returns the bound if the bound is violated
    """
    for i in range(0, len(candidate)):
        if candidate[i] < bounds[i][0]:
            candidate[i] = bounds[i][0]
        elif candidate[i] > bounds[i][1]:
            candidate[i] = bounds[i][1]
    

#### demonstrate CheckBounds

In [131]:
print(test[0][1])
test[0][1] = 15
print(test[0][1])
CheckBounds(test[0], bounds)
print(test[0][1])

25.94414285995579
15
20


In [57]:
def Mutate(population, popSize, bounds, mutationFactor, j):
    """
        performs mutation using the formula
        (x3 - x2) * mutation factor + x1
    """
    # select 3 random vectors from the population
    indices = list(range(0, popSize))
    indices.remove(j)
    
    # randIdx is now a 1x3 vector of indices
    randIdx = random.sample(indices, 3)

    x1 = population[randIdx[0]]
    x2 = population[randIdx[1]]
    x3 = population[randIdx[2]]

    # elementwise subtract (x2 - x3)
    temp = [x2i - x3i for x2i,x3i in zip(x2,x3)]

    # multiply (temp * mutationFactor) add to x1
    mutated = [tempi * mutationFactor + x1i for x1i,tempi in zip(x1, temp)]
    CheckBounds(mutated, bounds)
    
    return mutated

#### demonstrate Mutate

In [142]:
print(test[2])
mutated = Mutate(test, popSize, bounds, mutationFactor, 2)
print(mutated)

[8.150653108839833, 25.532830633865853, 0.5189086154466039]
[-1.525304882710472, 20, 2.572637527524336]


In [81]:
def Crossover(target, mutated, crossoverRate):
    """
        performs crossover between a target candidate and a mutated candidate
        if a random sample is less than the crossover rate for each variable
    """
    child = []
    for k in range(0, len(target)):
        if(random.random() <= crossoverRate):
            child.append(mutated[k])
        else:
            child.append(target[k])
    return child

#### demonstrate Crossover

In [145]:
child = Crossover(test[2], mutated, .75)
child

[-1.525304882710472, 20, 0.5189086154466039]

In [103]:
def Selection(objectiveFunction, child, current):
    """
        performs selection between two vectors
    """
    childScore = objectiveFunction(child)
    currentScore = objectiveFunction(current)
    if (childScore < currentScore):
        return child, childScore
    else:
        return current, currentScore

#### demonstrate Selection

In [146]:
print(test[2])
test[2], score = Selection(ObjectiveFunction, child, test[2])
print(test[2])
print(score)

[8.150653108839833, 25.532830633865853, 0.5189086154466039]
[-1.525304882710472, 20, 0.5189086154466039]
402.5958211364051


In [156]:
def DiffEvolv(objectiveFunction, bounds, numVars, popSize, mutationFactor, crossoverRate, numGenerations, verbose=False):
    """
        the differential evolution algorithm
    """
    # validate inputs
    if(len(bounds) is not numVars):
        print("ensure len(bounds) == numVars")
        return -1,-1
    if(mutationFactor > 2 or mutationFactor < 0):
        print("ensure mutationFactor is in range [0,2]")
        return -1,-1
    if(crossoverRate > 1 or corssoverRate < 0):
        print("ensure crossoverRate is in range [0,1]")
        return -1,-1
    if(popSize < 10):
        print("population size should be greater than 10")
        return -1,-1
    
    # create the initial population
    population = GeneratePopulation(bounds, numVars, popSize)

    # initialize the best candidate
    bestCandidate = population[0]
    
    # iterate through each generation
    for i in range(0, numGenerations):
        if(verbose):
            print("Generation: ", i)
        
        # initialize the scores for each generation
        scores = []
        
        # iterate through each candidate in the population
        for j in range(0, popSize):
            current = population[j]
            
            # perform mutation
            mutated = Mutate(population, popSize, bounds, mutationFactor, j)
            
            # perform crossover
            child = Crossover(current, mutated, crossoverRate)
            
            # perform selection
            population[j], score = Selection(objectiveFunction, child, current)
            
            # add to scores
            scores.append(score)
        
        bestFitness = min(scores)
        bestCandidate = population[scores.index(bestFitness)]
        if(verbose):
            print("current best: " + str(bestCandidate) + " with score: " + str(bestFitness))
    
    return bestCandidate, bestFitness

### Demonstrate entire algorithm

In [158]:
bounds = [(-10, 10), (-10, 10), (-10,10)]
numVars = 3
popSize = 20
mutationFactor = .4 # in range [0,2]
crossoverRate = .7
numGenerations = 100

In [159]:
solution, score = DiffEvolv(ObjectiveFunction, bounds, numVars, popSize, mutationFactor, crossoverRate, numGenerations)
solution

[-1.3125858246766127e-09, -2.981703515534781e-10, -2.4746283614122103e-10]

### What if the bounds are unknown and there are alot of variables?

In [165]:
bounds = [(-10000000, 10000000), (-10000000, 10000000), (-10000000, 10000000), (-10000000, 10000000), (-10000000, 10000000), (-10000000, 10000000), (-10000000, 10000000), (-10000000, 10000000), (-10000000, 10000000), (-10000000, 10000000)]
numVars = len(bounds)
popSize = 20
mutationFactor = .4 # in range [0,2]
crossoverRate = .7
numGenerations = 100
solution, score = DiffEvolv(ObjectiveFunction, bounds, numVars, popSize, mutationFactor, crossoverRate, numGenerations)
solution, score

([-1618.7233129853994,
  -11639.258173174752,
  -6898.286298098288,
  -8931.45248430592,
  -2122.8492989311567,
  -3125.927664601403,
  6965.299781676927,
  5617.49099043211,
  116189.33176960441,
  -4137.225438644771],
 13876876763.699646)

.


### increase the population size and number of generations!

In [166]:
bounds = [(-10000000, 10000000), (-10000000, 10000000), (-10000000, 10000000), (-10000000, 10000000), (-10000000, 10000000), (-10000000, 10000000), (-10000000, 10000000), (-10000000, 10000000), (-10000000, 10000000), (-10000000, 10000000)]
numVars = len(bounds)
popSize = 100
mutationFactor = .4 # in range [0,2]
crossoverRate = .7
numGenerations = 1000
solution, score = DiffEvolv(ObjectiveFunction, bounds, numVars, popSize, mutationFactor, crossoverRate, numGenerations)
solution, score

([5.182623024502645e-27,
  -2.772153693888324e-26,
  -4.921515693964067e-27,
  2.780743471461727e-26,
  -1.0172929921183497e-26,
  1.1279751447714034e-26,
  1.619818629511732e-26,
  2.4986496742484434e-27,
  -2.901246935485349e-26,
  -8.914973847203543e-27],
 3.013363855913711e-51)